In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
#%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')
#
import numpy as np
from tqdm import trange

#
#from visualize import Visualize


#
import sleap

#
import cv2
import os

#
from tqdm import tqdm
import parmap

#
import pandas as pd

#
from track.track import make_human_label_only_slp, make_human_label_only_video

Autosaving every 180 seconds


/home/cat/miniconda3/envs/sleap_new/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
################################################################
########### SET FNAMES FOR HUMAN AND ID SWITCH FILES ###########
################################################################
from track.track import DatabaseLoader

#
#root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/'
root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/'
fname_database = 'database.xlsx'
human_slp_dir = 'human_labels'
input_dir = 'input_dir'
n_cores = 32
network_type = "Night"   # "Day" or "Night"

# load database class
d = DatabaseLoader(root_dir + fname_database,
                   network_type)
d.nn_type = network_type
d.root_dir = root_dir
d.human_slp_dir = human_slp_dir
d.input_dir = input_dir
d.n_cores = n_cores
d.network_type = network_type
d.track_type = 'feature'
d.recompute_spine_centres = False
d.skip_id_switch_make = True

# hard code the # of animals and features for now
d.n_animals = 6
d.n_features = 6


# make id switch files over all files in /input_dir/ and database.xlsx 
d.make_id_switch_files()

# process the human labeled .slp file to short version
d.fname_slp_human_labels = 'cohort4_night.1000.slp'
d.clean_human_slp()

# merge the id switch and human files

d.subsample=2
d.merge_human_and_id_switches()

#
print ("DONE...")


334it [00:00, 8517.23it/s]
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


ALL SLP VIDS:  [['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_20_50_33_472153_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_21_57_28_031303_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_23_04_17_922045_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_00_11_09_854170_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_01_18_01_760467_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_02_24_56_195811_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_03_31_49_946989_compressed_Night_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input

making hybrid movie:   6%|███▍                                                      | 2/34 [00:05<01:12,  2.27s/it]

video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_23_04_17_922045_compressed.mp4


making hybrid movie:  18%|██████████▏                                               | 6/34 [00:11<00:45,  1.61s/it]

...

video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_30_23_01_18_531536_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_01_01_13_25_690487_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_01_23_14_29_288743_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_02_23_27_57_884608_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_03_06_04_21_479563_compressed.mp4


making hybrid movie: 100%|█████████████████████████████████████████████████████████| 34/34 [00:17<00:00,  1.97it/s]


video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_04_23_53_03_155670_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_05_02_05_11_593158_compressed.mp4
video has no switch frames, :  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_10_06_23_11_11_555079_compressed.mp4
Video nframes:  190
 # of slps found:  34
 # of vids found:  34
slp file:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_20_50_33_472153_compressed_Night_deleted.slp
slp file:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_21_57_28_031303_compressed_Night_deleted.slp
slp file:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_23_04_17_922045_compressed_Night_deleted.slp
slp file:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_00_1

total frames: 1000: 1000it [03:43,  4.48it/s]


# of images saved  1000
Video(filename=/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Night_merged.avi, shape=(None, None, None, None), backend=MediaVideo)
slp ctrl:  0
Loaded video sizes: width, height:  1160.0 830.0
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Night_human_labels.avi
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Night_id_switches_cropped.mp4
done...
DONE...


In [14]:
d = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_16_23_20_653413_compressed_Day.npy')

print (d.shape)

print (np.nanmin(d[:,:,:,0]))
print (np.nanmax(d[:,:,:,0]))
       
print (np.nanmin(d[:,:,:,1]))
print (np.nanmax(d[:,:,:,1]))
    
    

(86407, 6, 6, 2)
4.457507133483887
1172.404296875
2.651884078979492
825.70654296875


In [96]:

slps = np.load('/home/cat/meta_data_slp.npy',allow_pickle=True)
vids = np.load('/home/cat/meta_data.npy', allow_pickle=True)

for k in range(len(slps)):
    #print ('')
    temp = slps[k]
    temp2 = vids[k]
    if temp[1]!=temp2[1]:
        print (temp)
        print (temp2)



['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_27_17_48_48_476256_compressed_Day_deleted.slp'
 '0']
['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_27_17_48_48_476256_compressed_Day_deleted.slp'
 '42']


In [6]:

fname = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_22_31_26_256034_compressed_Night_deleted.slp'

sleap.load_file(fname)



Labels(labeled_frames=26, videos=1, skeletons=1, tracks=6)